In [1]:
import os

In [2]:
%pwd

'c:\\code\\ML\\time_series_forecast\\time-series-forecast\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\code\\ML\\time_series_forecast\\time-series-forecast'

In [5]:
#config yaml

In [6]:
#config entity
from dataclasses import dataclass
from pathlib import Path
@dataclass
class Model:
    data_path : Path
    report : Path
    model_save_path : Path


In [7]:
#configuration

In [8]:
from src.time_series.constants import *
from src.time_series.utils.common import create_directories , read_yaml
from src.time_series.entity.config_entity import DataIngestionconfig,EDA



class ConfigurationManager:
    def __init__(self,config_file_path=CONFIG_FILE_PATH,params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionconfig:
        config = self.config.data_ingestion
        data_ingestion_config = DataIngestionconfig(root_dir= config.root_dir,source_url=config.source_url,local_data_file=config.local_data_file)
        return data_ingestion_config
    
    def get_eda_config(self)->EDA:
        config = self.config.EDA
        create_directories([config.report_path])
        create_directories([config.data_output])
        eda_config = EDA(data_path= config.data_path,report_path=config.report_path,data_output=config.data_output)
        return eda_config
    
    def get_model_config(self)->Model:
        config = self.config.model
        create_directories([config.report])
        model_config = Model(data_path=config.data_path,report=config.report,model_save_path= config.model_save_path)
        return model_config

In [9]:
#component

In [10]:
pip install pmdarima

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
pip install prophet



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:
import os
import pandas as pd
from pmdarima import auto_arima
from prophet import Prophet 
import numpy as np
from sklearn.metrics import mean_squared_error
import joblib
import matplotlib.pyplot as plt
from src.time_series.logger import logger

class ModelBuildingComponent:
    def __init__(self, config):
        self.config = config

    def load_data(self):
        if not os.path.exists(self.config.data_path):
            raise FileNotFoundError(f"Input file not found: {self.config.data_path}")
        return pd.read_csv(self.config.data_path, parse_dates=['Month'], index_col='Month')

    def build_model(self):
        df = self.load_data()
        df.columns = ['passengers']

        # Split the data
        train = df[:-12]
        test = df[-12:]

        # ========== Auto ARIMA ==========
        auto_arima_model = auto_arima(
            train,
            start_p=0, start_q=0,
            max_p=3, max_q=3,
            seasonal=True,
            m=12,
            d=1, D=1,
            trace=True,
            error_action='ignore',
            suppress_warnings=True,
            stepwise=True
        )

        # Save ARIMA summary
        with open(os.path.join(self.config.report, 'data_report.txt'), 'a') as f:
            f.write("\n\n--- AUTO ARIMA ---\n")
            f.write(str(auto_arima_model.summary()))

        # Forecast & RMSE
        forecast_auto = auto_arima_model.predict(n_periods=12)
        forecast_auto = pd.Series(forecast_auto, index=test.index)

        rmse_auto = np.sqrt(mean_squared_error(test, forecast_auto))
        with open(os.path.join(self.config.report, 'data_report.txt'), 'a') as f:
            f.write(f"\nAuto ARIMA RMSE: {rmse_auto:.2f}\n")

        # Plot
        plt.figure(figsize=(10, 5))
        plt.plot(train, label='Train')
        plt.plot(test, label='Test')
        plt.plot(forecast_auto, label='Auto ARIMA Forecast', color='orange')
        plt.xlabel("Date")
        plt.ylabel("Passengers")
        plt.title("Auto ARIMA Forecast")
        plt.legend()
        plt.grid(True)
        plt.savefig(os.path.join(self.config.report, 'auto_arima_forecast.png'))
        plt.clf()

        # Save ARIMA model
        joblib.dump(auto_arima_model, filename=os.path.join(self.config.model_save_path, 'auto_arima.pkl'))
        logger.info("Auto ARIMA model saved.")

        # ========== Prophet ==========
        df_prophet = df.reset_index().rename(columns={'Month': 'ds', 'passengers': 'y'})
        train_prophet = df_prophet.iloc[:-12]
        test_prophet = df_prophet.iloc[-12:]

        prophet_model = Prophet(weekly_seasonality=False, yearly_seasonality=True)
        prophet_model.fit(train_prophet)

        future = prophet_model.make_future_dataframe(periods=12, freq='MS')
        forecast = prophet_model.predict(future)

        # Save Prophet model
        joblib.dump(prophet_model, os.path.join(self.config.model_save_path, 'prophet.pkl'))
        logger.info("Prophet model saved.")

        # Save forecast
        forecast.to_csv(os.path.join(self.config.report, 'forecast.csv'))

        # Evaluation
        forecast_test = forecast.set_index('ds').loc[test_prophet['ds']]
        forecast_series = forecast_test['yhat']
        rmse_prophet = np.sqrt(mean_squared_error(test_prophet['y'], forecast_series))

        with open(os.path.join(self.config.report, 'data_report.txt'), 'a') as f:
            f.write("\n\n--- PROPHET ---\n")
            f.write(f"PROPHET RMSE: {rmse_prophet:.2f}\n")

        # Forecast plot
        fig = prophet_model.plot(forecast)
        fig.suptitle("Prophet Forecast")
        fig.set_size_inches(10, 5)
        plt.xlabel("Date")
        plt.ylabel("Passengers")
        fig.savefig(os.path.join(self.config.report, 'prophet_forecast.png'))
        plt.clf()

        # Components plot
        fig2 = prophet_model.plot_components(forecast)
        fig2.set_size_inches(10, 8)
        fig2.savefig(os.path.join(self.config.report, 'prophet_components.png'))
        plt.clf()

        logger.info("Forecasts and plots saved.")


In [30]:
# pipeline

In [31]:
try:
    config = ConfigurationManager()
    model_config = config.get_model_config()
    model_build_comp = ModelBuildingComponent(config= model_config)
    model_build_comp.build_model()
except Exception as e:
    raise e


[2025-07-04 18:20:55,936: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-04 18:20:55,940: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-04 18:20:55,943: INFO: common: created directory at: artifacts]
[2025-07-04 18:20:55,947: INFO: common: created directory at: artifacts/model]
Performing stepwise search to minimize aic


c:\code\ML\time_series_forecast\time-series-forecast\venv\lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\code\ML\time_series_forecast\time-series-forecast\venv\lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(0,1,0)(1,1,1)[12]             : AIC=inf, Time=1.83 sec
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=905.065, Time=0.02 sec


c:\code\ML\time_series_forecast\time-series-forecast\venv\lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\code\ML\time_series_forecast\time-series-forecast\venv\lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(1,1,0)(1,1,0)[12]             : AIC=900.823, Time=0.23 sec


...

18:21:04 - cmdstanpy - INFO - Chain [1] done processing


[2025-07-04 18:21:04,704: INFO: model: Chain [1] done processing]
[2025-07-04 18:21:05,042: INFO: 4062261798: Prophet model saved.]
[2025-07-04 18:21:06,627: INFO: 4062261798: Forecasts and plots saved.]


<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x800 with 0 Axes>